In [ ]:
import time, DM_Profiles, numpy, scipy
import pynbody as pyn
from matplotlib import pylab as plt
s = pyn.load('/media/tengiz/81498267-fb7c-4587-88ea-5c63aaf2cb66/Pynbody/testdata/g15784.lr.01024.gz')
s.physical_units()
h = s.halos()

In [ ]:
def model_prep(halo):
    """
    This prepares profile for model to take in
    """
    # centering to generate profile and placing particles back 
    with pyn.analysis.angmom.faceon(h[halo], cen_size  =  '10 kpc'):
        r_200 = pyn.analysis.halo.virial_radius(h[halo].d, overden = 200)/pyn.array.SimArray(1, s['pos'].units)
        profile = pyn.analysis.profile.Profile(h[halo].d, min = 2*max(h[halo].d['eps']), max = r_200, ndim = 3, type = 'log', nbins = 40)
    
    # calculating hubble constant in units from (https://www.pnas.org/content/pnas/95/11/5956.full.pdf)
    H0 = pyn.analysis.cosmology.H(s)
    factor = pyn.array.SimArray(100., units = 'km s**-1 Mpc**-1')
    factor.convert_units(H0)
    hubble = 1/float(factor)
    
    # calculating steallar and halo mass
    
    sm = h[halo].s['mass'].sum()
    hm = h[halo].d['mass'].sum()
        
    return profile, hubble, sm, hm

In [ ]:
def den_plot(halo_profile, to_save = False):
    '''
    takes model profile object from DM_Profiles and plots den profile with a curve_fit
    '''
    
    fig, ax = plt.subplots() 
    ax.plot(halo_profile.radii, numpy.power(10, halo_profile.log_den), 'g.')
    ax.plot(halo_profile.radii, numpy.power(10, halo_profile.log_rho(numpy.array(halo_profile.radii), *halo_profile.params)), 'r-')
    ax.grid()
    ax.legend(('data','fit'))
    ax.set_title(halo_profile.name + ' ' + halo_profile.pmodel + ' density profile')
    ax.set_xlabel('$R$ [kpc]')
    ax.set_ylabel(r'$\rho$ [M$_{\odot}$ /kpc$^{3}$]')
    plt.errorbar(halo_profile.radii, numpy.power(10, halo_profile.log_den) , yerr = numpy.array(halo_profile.den_error), fmt = 'none')
    ax.set_yscale('log')
    ax.set_xscale('log')
    if to_save:
        plt.savefig('../Graphs/density_plot' + haloprofile.name +  '_density.jpg')
    plt.show()

In [ ]:
halo = 1
start_time = time.time()
p,hubble, sm, hm = model_prep(halo) 

In [ ]:
hp = DM_Profiles.model(p, 'halo_' +  str(halo), h = hubble, stellar_mass = sm, halo_mass = hm, pmodel = 'Einasto')
print(hp.output())
den_plot(hp)
print(time.time() - start_time)